In [1]:
# load the dataset
import pandas as pd
import numpy as np
import os

signals = []

for file in os.listdir('intertial_signals'):
    data = pd.read_csv(os.path.join('intertial_signals', file), sep=r"\s+", header=None)
    signals.append(data)

data = np.stack(signals, axis=-1)
y = pd.read_csv('y_train.txt', sep=r"\s+", header=None).squeeze()
subjects = pd.read_csv('subject_train.txt', sep=r"\s+", header=None).squeeze()

In [2]:
# subject disjoint split

unique = np.unique(subjects)
np.random.seed(42)

train_subjects = np.random.choice(unique, size=int(0.8 * len(unique)), replace=False)
val_subjects = np.setdiff1d(unique, train_subjects)

train_mask = np.isin(subjects, train_subjects)
val_mask = np.isin(subjects, val_subjects)

X_train = data[train_mask]
y_train = y[train_mask]

X_val = data[val_mask]
y_val = y[val_mask]

overlap = set(train_subjects).intersection(set(val_subjects))
print("Overlap count:", len(overlap))

Overlap count: 0


In [5]:
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier

# Reshape data for MLP
n_samples, n_timesteps, n_features = X_train.shape
X_train_r = X_train.reshape(n_samples, n_timesteps * n_features)

n_samples_val, n_timesteps_val, n_features_val = X_val.shape
X_val_r = X_val.reshape(n_samples_val, n_timesteps_val * n_features_val)

mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=30, random_state=42)

mlp.fit(X_train_r, y_train)
y_pred_mlp = mlp.predict(X_val_r)

print(classification_report(y_val, y_pred_mlp))

              precision    recall  f1-score   support

           1       0.96      0.79      0.87       291
           2       0.68      0.99      0.81       256
           3       1.00      0.73      0.84       239
           4       0.69      1.00      0.81       299
           5       1.00      0.58      0.74       318
           6       1.00      1.00      1.00       320

    accuracy                           0.85      1723
   macro avg       0.89      0.85      0.84      1723
weighted avg       0.89      0.85      0.85      1723



C:\Users\taylon4\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  warnings.warn(
